# Import packages
run the following cell to import necessary packages to run the analysis

In [30]:
import os
from pathlib import Path
from tqdm.notebook import tqdm
from bioio import BioImage
import bioio_czi
from pylibCZIrw import czi as pyczi

# Reading data

In [2]:
data_path = Path('/media/npmartins/Data/nuno_martins/Image_data/NPM_Ex004_Re04/')
filelist = os.listdir(data_path)
print(len(filelist))

22


In [3]:
filelist = [x for x in filelist if x.find('.czi')>0]
filelist.sort()

In [34]:
file_index = len(filelist)-1
file_path = str(data_path)+'/'+filelist[file_index]
filename = filelist[file_index][:filelist[file_index].index('.czi')]
print(file_path)
print(filename)

/media/npmartins/Data/nuno_martins/Image_data/NPM_Ex004_Re04/NPM_Ex004_Re04_hexa2_gel2_tiles_20x.czi
NPM_Ex004_Re04_hexa2_gel2_tiles_20x


In [36]:
# save_path = str(data_path)+'/tiles/'
parent_save_path = Path(str(data_path)+'/tiles/')
parent_save_path.mkdir(parents=True, exist_ok=False)

___
# Read raw data
reading raw tile data from `czi` tile without stitching for visualization.

In [6]:
img = BioImage(file_path, reader=bioio_czi.Reader, reconstruct_mosaic=False)
img.dims, img.dims.order

(<Dimensions [M: 20, T: 1, C: 4, Z: 1, Y: 512, X: 512]>, 'MTCZYX')

In [19]:
img.shape, len(img.scenes)

((20, 1, 4, 1, 512, 512), 15)

In [21]:
# img.scenes
for scene in img.scenes:
    img.set_scene(scene)
    print(img.dims)

<Dimensions [M: 20, T: 1, C: 4, Z: 1, Y: 512, X: 512]>
<Dimensions [M: 9, T: 1, C: 4, Z: 1, Y: 512, X: 512]>
<Dimensions [M: 26, T: 1, C: 4, Z: 1, Y: 512, X: 512]>
<Dimensions [M: 23, T: 1, C: 4, Z: 1, Y: 512, X: 512]>
<Dimensions [M: 20, T: 1, C: 4, Z: 1, Y: 512, X: 512]>
<Dimensions [M: 16, T: 1, C: 4, Z: 1, Y: 512, X: 512]>
<Dimensions [M: 33, T: 1, C: 4, Z: 1, Y: 512, X: 512]>
<Dimensions [M: 21, T: 1, C: 4, Z: 1, Y: 512, X: 512]>
<Dimensions [M: 81, T: 1, C: 4, Z: 1, Y: 512, X: 512]>
<Dimensions [M: 23, T: 1, C: 4, Z: 1, Y: 512, X: 512]>
<Dimensions [M: 31, T: 1, C: 4, Z: 1, Y: 512, X: 512]>
<Dimensions [M: 8, T: 1, C: 4, Z: 1, Y: 512, X: 512]>
<Dimensions [M: 12, T: 1, C: 4, Z: 1, Y: 512, X: 512]>
<Dimensions [M: 23, T: 1, C: 4, Z: 1, Y: 512, X: 512]>
<Dimensions [M: 15, T: 1, C: 4, Z: 1, Y: 512, X: 512]>


In [22]:
def getDimensionPosition(dimension_order, dimension_name):
    return dimension_order.index(dimension_name)

In [48]:
for scene in tqdm(img.scenes):
    img.set_scene(scene)
    # print(scene)
    # print(img.shape)

    save_path = Path(str(parent_save_path)+'/'+filename+'_'+str(scene)+'/')
    save_path.mkdir(parents=True, exist_ok=True)
    
    image_data = img.data
    n_tiles = image_data.shape[getDimensionPosition(img.dims.order, 'M')]
    for n in range(n_tiles):
        tile = image_data[n]
        
        tile_save_name = str(save_path)+'/'+filename+'_'+str(scene)+"_"+str(n+1).zfill(2)+'.czi'
        with pyczi.create_czi(tile_save_name, exist_ok=True) as czidoc_w:
            for t in range(image_data.shape[getDimensionPosition(img.dims.order, 'T')]):
                for c in range(image_data.shape[getDimensionPosition(img.dims.order, 'C')]):
                    for z in range(image_data.shape[getDimensionPosition(img.dims.order, 'Z')]):
                        czidoc_w.write(
                            data=tile[t, c, z, ...],
                            plane={"T": t, "C": c, "Z": z}
                        )


  0%|          | 0/15 [00:00<?, ?it/s]